In [1]:
#!kill -9 -1 #reiniciando la maquina virtual

In [2]:
#!pip list
!nvcc --version  #Version de CUDA en la maquina virtual

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [3]:
import tensorflow as tf
import timeit               #para medir tiempos
import numpy as np
import pandas as pd 
import os
import time
import sys

In [4]:
print("Tensorflow Version: ", tf.__version__)
print("Dispositivos disponibles para entrenar: ", tf.config.list_physical_devices())
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Encontrada la GPU: {}'.format(device_name))

Tensorflow Version:  2.7.0
Dispositivos disponibles para entrenar:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Encontrada la GPU: /device:GPU:0


In [5]:
def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)   

In [6]:
cpu()  #ejecutamos entrenamiento con CPU
gpu()  #ejecutamos entrenamiento con GPU
# Run the op several times.
print('TIEMPO (seg) para entrenar una red convolucional de 32x7x7x3 filtros sobre un randomico de 100x100x100x3 imagenes '
      '(batch x height x width x channel). suma de 10 epochs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

TIEMPO (seg) para entrenar una red convolucional de 32x7x7x3 filtros sobre un randomico de 100x100x100x3 imagenes (batch x height x width x channel). suma de 10 epochs.
CPU (s):
3.7016485500007548
GPU (s):
0.054879336999874795
GPU speedup over CPU: 67x


In [7]:
#tf.device('/gpu:0') #activando la CPU
tf.device('/device:GPU:0') #activando la GPU 

In [8]:
fileDL= tf.keras.utils.get_file('ElGatoConBotas.txt','https://raw.githubusercontent.com/GustavoAdolfoGuizaWalteros/Deep_Learning/main/Cuentos_txt/ElGatoConBotas.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')


16384/7808 [==============================================================] - 0s 0us/step


In [9]:
import re
from unicodedata import normalize
#pasa todo a minuscula
texto     = texto.lower()
#reemplazar tildes por letras similares sin tildes
transfor  = dict.fromkeys(map(ord, u'\u0301\u0308'), None)
texto     = normalize('NFKC', normalize('NFKD', texto).translate(transfor))
#quitar saltos de linea
texto      = texto.strip()
texto      = re.sub('\r|\n', ' ',texto)
#quitar espacios dobles
texto      = re.sub(' +', ' ', texto)
#quitando caracteres especiales
texto = re.sub(r"[^a-zA-Z0-9]+"," ",texto)
print(texto)

el gato con botas un molinero dejo como unica herencia a sus tres hijos su molino su burro y su gato el reparto fue bien simple ya que no se necesito llamar ni al abogado ni al notario pueshabrian consumido por el cobro todo el pobre patrimonioel mayor recibio el molino el segundo se quedo con el burro y al menor le toco solo elgato este se lamentaba de su misera herencia fdlis hermanos decia podran ganarse la vida convenientemente trabajando juntos lo que es yo despues de comerme a mi gato y de hacerme un manguito con su piel me morirede hambre el gato que escuchaba estas palabras pero se hacia el desentendido le dijo en tono serio y pausado no debes afligirte mi senor solo tienes que proporcionarme una bolsa y un par de botaspara andar por entre los matorrales y veras que tu herencia no es tan pobre como piensas aunque el amo del gato no abrigaba sobre esto grandes ilusiones aunque le habiavisto dar tantas muestras de agilidad para cazar ratas y ratones colgarse de los pies esconders

In [10]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:7808 caracteres
el texto esta compuesto de estos :25 caracteres
[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z']


In [11]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#-----------revisando las conversiones

#for char,_ in zip(char2idx, range(len(vocab))):
    #print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: 'el gato con botas un molinero dejo como unica herencia a sus tres hijos su molino su burro y su gato'
array([ 5, 11,  0,  7,  1, 19, 14,  0,  3, 14, 13,  0,  2, 14, 19,  1, 18,
        0, 20, 13,  0, 12, 14, 11,  9, 13,  5, 17, 14,  0,  4,  5, 10, 14,
        0,  3, 14, 12, 14,  0, 20, 13,  9,  3,  1,  0,  8,  5, 17,  5, 13,
        3,  9,  1,  0,  1,  0, 18, 20, 18,  0, 19, 17,  5, 18,  0,  8,  9,
       10, 14, 18,  0, 18, 20,  0, 12, 14, 11,  9, 13, 14,  0, 18, 20,  0,
        2, 20, 17, 17, 14,  0, 23,  0, 18, 20,  0,  7,  1, 19, 14])


In [12]:
rows=[]
columns=['num','vocab']
for i, voc in enumerate(vocab):
  #print(i,'-->', voc)
  rows.append([i,voc])
df= pd.DataFrame(columns=['num','vocab'],data=rows)
df.head(10)
df.to_csv('data_vocab.csv',index=False)

In [13]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=150
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'el gato con botas un molinero dejo como unica herencia a sus tres hijos su molino su burro y su gato el reparto fue bien simple ya que no se necesito l'
'lamar ni al abogado ni al notario pueshabrian consumido por el cobro todo el pobre patrimonioel mayor recibio el molino el segundo se quedo con el burr'
'o y al menor le toco solo elgato este se lamentaba de su misera herencia fdlis hermanos decia podran ganarse la vida convenientemente trabajando juntos'
' lo que es yo despues de comerme a mi gato y de hacerme un manguito con su piel me morirede hambre el gato que escuchaba estas palabras pero se hacia e'
'l desentendido le dijo en tono serio y pausado no debes afligirte mi senor solo tienes que proporcionarme una bolsa y un par de botaspara andar por ent'
're los matorrales y veras que tu herencia no es tan pobre como piensas aunque el amo del gato no abrigaba sobre esto grandes ilusiones aunque le habiav'
'isto dar tantas muestras de agilidad para cazar ratas y ratones colgarse de

In [14]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  'el gato con botas un molinero dejo como unica herencia a sus tres hijos su molino su burro y su gato el reparto fue bien simple ya que no se necesito '
Target data:  'l gato con botas un molinero dejo como unica herencia a sus tres hijos su molino su burro y su gato el reparto fue bien simple ya que no se necesito l'


In [15]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 64
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

<MapDataset shapes: ((150,), (150,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 150), (64, 150)), types: (tf.int64, tf.int64)>


In [16]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def lossy(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def create_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  #En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
  #con los argumentos por defecto del optimizador Adam. 
  model.compile(optimizer='adam',
              loss=lossy,
              metrics=['accuracy'])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=  -La capa LSTM consta más de 5 millones de parametros)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           6400      
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 25)            25625     
                                                                 
Total params: 5,279,001
Trainable params: 5,279,001
Non-trainable params: 0
_________________________________________________________________


In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [18]:
checkpoint_dir='/content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Redes neuronales recurrentes (RNN)/checkpointsV5/'
checkpoint_prefix= os.path.join(checkpoint_dir,"cp_{epoch:04d}.ckpt")


cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                               monitor='loss',
                                               verbose=1,
                                               save_weights_only=True,
                                               save_best_only=True,
                                               mode='auto')


In [19]:
EPOCHS=100
history=model.fit(dataset, 
                  epochs=EPOCHS, 
                  verbose=1,
                  callbacks=[cp_callback])

Epoch 1/100


ValueError: ignored


#####4.2a-1 entrenando desde un checkpoint

---
Desde la carpeta que optamos guardar los checkpoints

*   el archivo .data es el archivo que contiene nuestras variables de entrenamiento y vamos a ir tras él.
*   el archivo checkpoint, simplemente mantiene un registro de los últimos archivos de punto de control guardados




In [ ]:
#creamos un modelo con iguales caracteristicas al 1° modelo
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)

#buscamos el ultimo checkpoint de entrenamiento
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

In [ ]:
# cargamos los pesos al nuevo modelo (estos valores tienes una variación de un 10%)
model.load_weights(latest)
# continuamos el entrenamiento desde el checkpoint en que quedamos
history2=model.fit(dataset, 
                    epochs=50, 
                    verbose=1,
                    callbacks=[cp_callback])

####P4.2b entrenando con tensorboard (opcional)

#####Activando TENSORBOARD 

---
(DASHBOARD para ver el proceso de entrenamiento)

In [ ]:
# You can change the directory name
LOG_DIR = 'tb_logs'

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

import os
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)
  
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR))

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, 
                         histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=BATCH_SIZE,
                         write_images=True)

#####Fit

In [ ]:

#model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)
history_TB=model2.fit(dataset, 
                    epochs=200, 
                    verbose=1,
                    callbacks=[tbCallBack])

##P5. Generando texto nuevo usando la RNN

In [ ]:
#creamos un modelo tomando como base el ultimo checkpoint
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [ ]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.2  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))


###P5.1 generando texto 

In [ ]:
print(generate_text(model, start_string=u"la colonia europea"))

##P6.exportando modelo

---
Guardamos y Serializamos el Modelo (con esto ya podemos vender nuestro modelo de predicción de texto según lo aprendido por nuestra RNN).


In [ ]:
from keras.models import model_from_json
import os
dir_export= '/content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Redes neuronales recurrentes (RNN)/Modelos'
#dir_export= os.path.join(dir_drive)
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open(os.path.join(dir_export,'RNN_LasMinasDelReySalomon_json.json'), 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(dir_export,'RNN_LasMinasDelReySalomon_pesos.hdf5'))
model.save(os.path.join(dir_export,'RNN_LasMinasDelReySalomon_model.h5'))
print("modelo salvado en Drive de google")

##P7.Cargando un modelo serializado

###P7.1 descargamos el modelo usando wget

In [ ]:
!wget https://github.com/michelBolivar/Deep_learning/blob/main/Corte2/cuento/cuento_1500/modelRNN_cuentos2_2.h5?raw=true \
      -O modelRNN_cuentos2_2.h5

####P7.1a descargamos el modelo usando PYRIND & URLLIB (OPCIONAL)

In [ ]:
!pip install pyprind

In [ ]:
def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size
    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d segundos transcurrido" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()

import urllib.request
url_github_Model='https://github.com/luisFernandoCastellanosG/Machine_learning/blob/master/DeepLearning/PLN/recurrent_network_RNN/Modelos/RNN_LasMinasDelReySalomon_model.h5?raw=true'
urllib.request.urlretrieve(url_github_Model,
                           'RNN_LasMinasDelReySalomon_model2.h5', 
                           reporthook)

###P7.2 instanciamos el modelo descargado

In [ ]:
new_model = tf.keras.models.load_model('/content/modelRNN_cuentos2_2.h5')

In [ ]:
df2 = pd.read_csv("https://raw.githubusercontent.com/luisFernandoCastellanosG/Machine_learning/master/DeepLearning/PLN/recurrent_network_RNN/Modelos/data_vocab.csv")
df2.head()

In [ ]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval  = [char2idx[s] for s in start_string]
  input_eval  = tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.2  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))


In [ ]:
print(generate_text(new_model, start_string=u"los fantasmas de "))